In [1]:
import pandas as pd
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import re
import string
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans
import os

In [2]:
i = 1
o = 1
def stripping(txt):
    txt = BeautifulSoup(txt)
    code = [str(s.extract()) for s in txt('code')]
    txt = txt.get_text()
    txt.replace("\n","")
    txt = txt.translate(str.maketrans('', '', string.punctuation))
    txt = txt.lower()
    return txt
def pstripping(txt):
    txt = BeautifulSoup(txt)
    txt = txt.get_text()
    txt.replace("\n","")
    txt = txt.translate(str.maketrans('', '', string.punctuation))
    txt = txt.lower()
    return txt
def code(txt):
    txt = BeautifulSoup(txt)
    code = [str(s.extract()) for s in txt('code')]
    code = [i.replace("<code>","") for i in code]
    code = [i.replace("</code>","") for i in code]
    code = " ".join(code)
    return code
def get_labels(n):
    global i
    print()
    print(n)
    print("\n","Enter label for sample ",i)
    print()
    o = int(input())
    print("---------------------------------------------------")
    i+=1

In [ ]:
daf = pd.read_csv("QueryResultsFinal.csv")

In [ ]:
daf["Total"] = daf["Title"] + daf["Body"] + daf["Accepted Answer"]
daf["Body Code"] = daf["Body"].apply(code) 
daf["Answer Code"] = daf["Accepted Answer"].apply(code)
daf["Total"] = daf["Total"].apply(stripping)
vec = TfidfVectorizer( strip_accents = "unicode", stop_words = "english")
transformed= vec.fit_transform(daf["Total"])
clusters = 12
km = KMeans(n_clusters=clusters, init='k-means++', max_iter=25, n_init=1,verbose=1,random_state=42)
km.fit(transformed)

In [ ]:
km = KMeans(n_clusters=clusters, init='k-means++', max_iter=250, n_init=1,verbose=1,random_state=42)
km.fit(transformed)

In [ ]:
'''daf["Group"] = km.predict(transformed)
for i in range(0,12):
    r = daf[daf["Group"]==i]["Total"]
    print("GROUP-",i,"\n",r.values[100],"\n-----",r.values[-101],"\n-----",r.values[-102])
    print("========================================================================================\n\n\n") '''

In [ ]:
daf,lb = tts(daf,test_size=0.0025,random_state=42)

In [ ]:
rt = daf[daf["Group"].isin([1,2,3,4,5,6,7,8,9])]
rt,lb = tts(rt,test_size=0.00525,random_state=42)
daf = pd.concat([daf,lb,lb]).drop_duplicates(keep=False)


In [3]:
labpath = "labelled_data" 
if not os.path.exists(labpath):
    os.makedirs(labpath)
unlabpath = "unlabelled_data"
if not os.path.exists(unlabpath):
    os.makedirs(unlabpath)

In [5]:
unlabfile = unlabpath+"/unlabelled_data.csv"
labfile = labpath + "/labelled_data.csv"
#daf.to_csv(unlabfile)
#lb.to_csv(labfile)


In [6]:
lb = pd.read_csv(labfile)


In [27]:
index = 82
o = 1
def write_labels(txt):
    global index
    global o 
    while o<82:
        o+=1
        return 0
    print("\n")
    print("-----------------------------------------------")
    print(txt,"\nEnter label for sample:","index ",index)
    n = int(input())
    f = labpath+"/"+ str(index) +".txt"
    if not os.path.isfile(f) or os.stat(f).st_size == 0:
        with open(f,'w') as file:
            file.write(str(n))
            file.close()
        index+=1
    else:
        print("Already there\n")
        index+=1
        return -1
    print("\n")
   

In [ ]:
lb["Total"] = lb["Total"].apply(pstripping)

### lb["Total"].apply(write_labels)

In [11]:
lb.head()

,Unnamed: 0,id,Title,Body,Tags,score,Answer ID,Accepted Answer,User Reputation,Total,Body Code,Answer Code,Group
0,49900,52325662,Run Django on Oracle Application Container,<p>I have an existing django project which I a...,<python><django><bash><oracle><oracle-cloud-in...,0,52473559,<p>With some more help from Oracle support I m...,1,run django on oracle application containeri ha...,ACCS[web.1]: Content of APP_HOME [ /u01/app/ ]...,ALLOWED_HOSTS settings.py ALLOWED_HOSTS = ['*'...,8
1,27371,33846060,matching indexes in python to categorize a column,<p>I have a column in a dataframe of length 10...,<python><match><dataframe>,0,33846541,<p>You could use <code>merge</code> and do som...,3,matching indexes in python to categorize a col...,1. bakery commercial\n 2. hospital ...,merge import pandas as pd\ndf = pd.merge( lef...,9
2,42364,49007929,What is taking so much memory?,"<p>I have a simple code, which reads csv file,...",<python><python-2.7><csv><memory><set>,2,49008391,<p>you could create your own hash function to ...,13,what is taking so much memoryi have a simple c...,"def my_func():\n area = ""W09""\n\n inf = ...",tuple hash = x*10**7 + y\n x y hash tuple 10**...,3
3,13793,21358309,Python 3.3 xlrd 0.9.2 on mac OS X 10.9.1. - Un...,<p>I'm a complete Python beginner and I wanted...,<python><python-3.3><xlrd>,1,21377830,<p>The last part of error message is saying th...,8,python 33 xlrd 092 on mac os x 1091 unable to...,"Python 3.3.3 (v3.3.3:c3896275c0f6, Nov 16 2013...",FileNotFoundError: [Errno 2] No such file or d...,3
4,12619,22264150,Instances calling method seem to reference sam...,<p>I am working through Cravens pygame tutoria...,<python><methods><pygame><self><instances>,0,22272298,<p>You put the <code>snow_list = []</code> var...,14,instances calling method seem to reference sam...,import pygame\nimport random\n\n# Define some ...,snow_list = [] snow_list = [] def __init__():,1


In [29]:
txt = BeautifulSoup("<code>Hello there!</code>")

In [32]:
2

<code>Hello there!</code>